In [1]:
import pandas as pd
import json
import re
import numpy as np
import os


## Data parsing

In [2]:
# Loading Data
results_filepath = "db_06_07_21_resp.json"
with open(results_filepath) as f:
    # remove image0/image1 vars since it prevents proper df merging
    g = re.sub(r"_image[0-3]", "", f.read())
    data = json.loads(g)
    df = pd.json_normalize(data)
    df.drop(labels=["__v", "_id.$oid"], axis=1, inplace=True)

# hacky solution to remove the panda-appended user_. entry
renamed = [name.split("_.")[-1] for name in df.columns]
col_rename = {i: j for i, j in zip(df.columns, renamed)}
df = df.rename(columns=col_rename)

df.loc[[0]]


,id,userProfiling_age,userProfiling_position,userProfiling_useOfAI,userProfiling_useOfDP,userProfiling_mlFamiliarity,saliencyMaps_globalSaliency_understandability,saliencyMaps_globalSaliency_usability,saliencyMaps_globalSaliency_informativeness,saliencyMaps_globalSaliency_value,...,userProfiling_useOfAI_details,saliencyMaps_globalSaliency_comments,saliencyMaps_localSaliency_comments,conceptAttribution_textAttributes_comments,trustScores_borderlineCases_comments,userProfiling_aiFamiliarity,userProfiling_comments,counterfactuals_twoAxisCounterfactuals_comments,counterfactuals_prototypeInterpolation_comments,userProfiling_position-Comment
0,1.623319e+12,30-40,Assisting physician (Assistenzarzt) for pathol...,in routine diagnostics,in routine diagnostics,1,5,5,6,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
user_df = df[
    [
        "userProfiling_age",
        "userProfiling_position",
        "userProfiling_useOfDP",
        "userProfiling_useOfAI",
        "userProfiling_useOfAI_details",
        "userProfiling_mlFamiliarity",
    ]
]

fields = ["Understandability", "Usability", "Informativeness", "Value"]

instance_identifiers = {
    "Counterfactuals: 1-axis" : "counterfactuals_prototypeInterpolation",
    "Counterfactuals: 2-axis" : "counterfactuals_twoAxisCounterfactuals",
    "Saliency Maps: Local" : "saliencyMaps_localSaliency",
    "Saliency Maps: Global" : "saliencyMaps_globalSaliency",
    "Concept Attribution: Text" : "conceptAttribution_textAttributes",
    "Prototypes" : "prototypes_prototypes",
    "Trust Scores: Borderline Cases" : "trustScores_borderlineCases",
}

result_dataframes = dict()
for name, id_ in instance_identifiers.items():
    result_dataframes[name] = df[
        [f"{id_}_{field.lower()}" for field in fields]
    ]
    result_dataframes[name].columns = fields
result_dataframes["Counterfactuals: 1-axis"]


,Understandability,Usability,Informativeness,Value
0,6,6,7,6
1,7,7,7,7
2,4,6,4,5
3,6,4,5,5
4,3,2,2,1
5,7,7,7,7
6,2,2,2,2
7,7,4,6,5
8,5,6,5,5
9,6,6,5,5


## Stacked diverging bar charts

In [19]:
from typing import Optional
from collections import Counter

import plotly
import plotly.graph_objects as go

def stackedBarChartDF(
    df: pd.DataFrame,
    title: str, 
    palette: list,
    attributes: list = ["Understandability", "Usability", "Informativeness", "Value"],
    labels: Optional[list] = None,
    save_fig: bool = False,
    save_dir: str = "images",
):
    counters = [Counter(df[attribute]) for attribute in attributes]

    fig = go.Figure()  # type: ignore
    
    category_order = [
        "Strongly disagree",
        "Disagree",
        "Slightly disagree",
        "Neutral",
        "Slightly agree",
        "Agree",
        "Strongly agree",
    ]

    number_order = [1, 2, 3, 4, 5, 6, 7]

    def add_bar_trace(num, transform):
        rating = number_order[num]
        counts = [counter[rating] for counter in counters]
        total_count = sum(counters[0].values())

        xvals = [transform(count) / total_count for count in counts]

        fig.add_trace(
            go.Bar(
                x=xvals,
                y=labels if labels else attributes,
                orientation="h",
                name=category_order[num],
                # customdata=xvals,
                # hovertemplate = "%{y}: %{customdata}",
                width=0.8,
                marker_color=palette[num],
            ) # type: ignore
        )  

    # negative side
    add_bar_trace(3, lambda x: x * -0.5)
    for num in reversed(range(0, 3)):
        add_bar_trace(num, lambda x: x * -1)

    # positive side
    add_bar_trace(3, lambda x: x * 0.5)
    for num in range(4, 7):
        add_bar_trace(num, lambda x: x)

    fig.update_layout(
        barmode="relative",
        yaxis_autorange="reversed",
        title=title,
        title_x=0.5,
        xaxis={"tick0": 0, "title": "Frequency", "tickformat": "%", "range": [-1, 1]},
        showlegend=False,
    )
    fig.show()

    if save_fig:
        if not os.path.exists(save_dir):
            os.mkdir(save_dir)
        filename = "".join(i for i in name if i not in "\\/:*?<>|")
        fig.write_image(f"{save_dir}/{filename}.png")

palette = [plotly.colors.diverging.RdBu[n] for n in [2, 3, 4, 5, 6, 7, 8]]  # type: ignore
space = ' ' * 4
labels = [
    f"I find the explanation{space}<br>intuitively understandable{space}", 
    f"The explanation helps me to understand{space}<br>factors relevant to the algorithm{space}",
    f"The explanation helps me to decide whether{space}<br> I can trust the generated annotations{space}",
    f"The explanation provides me with{space}<br>valuable information for my work{space}",
    ]

for name, df in result_dataframes.items():
    stackedBarChartDF(
        df,
        name,
        palette,
        labels=labels,
        save_fig=True,
        save_dir=os.path.splitext(results_filepath)[0],
    )


## Overall comparison

In [9]:
from itertools import cycle

def boxPlotDF(df, title):
    # throw out non-numeric data
    # set the color palette
    palette = cycle(plotly.colors.sequential.Viridis)  # type: ignore

    # iterate over columns and show average spread
    fig = go.Figure()  # type: ignore
    for column in df:
        fig.add_trace(
            go.Box(y=df[column], name=column, marker_color=next(palette)) # type: ignore
        )  

    fig.update_layout(
        title=title,
        title_x = 0.5,
        yaxis=dict(dtick=1),
        yaxis_range=[1, 7],
        yaxis_title="Average Rating",
        showlegend=False,
    )

    fig.show()


aggregated_df = pd.DataFrame()

for name, df in result_dataframes.items():
    aggregated_df[name] = df.apply(np.mean, axis=1)

sorted_index = aggregated_df.median().sort_values(ascending=False).index

boxPlotDF(aggregated_df[sorted_index], "Comparison of Annotation Average")


## User profiling

In [6]:
user_df


,userProfiling_age,userProfiling_position,userProfiling_useOfDP,userProfiling_useOfAI,userProfiling_useOfAI_details,userProfiling_mlFamiliarity
0,30-40,Assisting physician (Assistenzarzt) for pathol...,in routine diagnostics,in routine diagnostics,NaN,1
1,41-50,Researcher in pathology/neuropathology,in research,in research,"from MindPeak (BreastIHC), from VMscope (Cogni...",3
2,30-40,Researcher in pathology/neuropathology,[in research],[in research],NaN,7
3,41-50,Technician (MTA) for pathology/neuropathology,[in research],[in routine diagnostics],NaN,7
4,30-40,Trainee (Assistenzarzt) in pathology/neuropath...,[in research],[in research],QuPath immunohistochemistry positive cell dete...,2
5,30-40,Consultant (Facharzt) for pathology/neuropatho...,[in research],[in research],Aiforia,7
6,51-60,Researcher in pathology/neuropathology,[none],[none],NaN,2
7,41-50,Consultant (Facharzt) for pathology/neuropatho...,[in routine diagnostics],[none],NaN,4
8,41-50,Consultant (Facharzt) for pathology/neuropatho...,[none],[none],NaN,5
9,51-60,Consultant (Facharzt) for pathology/neuropatho...,"[in research, in routine diagnostics]","[in routine diagnostics, in research]",Roche Diagnostics,4
